In [1]:
import ee


In [3]:
import geemap.core as geemap


In [5]:
# Authenticate the Earth Engine API
ee.Authenticate()

True

In [7]:
# Initialize the Earth Engine API
ee.Initialize(project='ee-sandalusankaja2001')

In [9]:
print(ee.String('Hello from the Earth Engine servers!').getInfo())


Hello from the Earth Engine servers!


In [11]:
# Define the area of interest (random urban area coordinates, e.g., New York City)
aoi = ee.Geometry.Point([-74.006, 40.7128]).buffer(5000)  # 5km buffer around the point

In [13]:
# Load Sentinel-2 Image Collection and filter by date and cloud cover
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Get a random image from the collection
random_image = sentinel2.randomColumn().first()

In [15]:

# Create a map centered on the area of interest
Map = geemap.Map(center=[40.7128, -74.006], zoom=12)

In [17]:
# Add Sentinel-2 image to the map
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # RGB bands
}
Map.addLayer(random_image, vis_params, 'Random Urban Area')

In [19]:
# Display the map
Map


Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [21]:
# Define an example image.
img = ee.Image.random()

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(83.277, 17.7009, 12)
m.add_layer(dataset.mean(), visualization, 'RGB')

In [23]:
# Add the image to the map.
m.add_layer(img, None, 'Random image')

NameError: name 'm' is not defined

In [ ]:
# Display the map (you can call the object directly if it is the final line).
display(m)